In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.svm import SVC
import sys
import scipy.io as sio
import os
import openpyxl
import re
import pickle
import time

In [ ]:
import keras
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from keras import models
from keras.layers import Dense

In [ ]:
num_classes = 3
channels=62
drive='E'
entropy='diff_entropy'

In [ ]:
mypath=drive+r'feature matrix address'
file = os.listdir(mypath)
file

In [ ]:
label_file='label address'
label_fname = os.path.join(mypath,label_file)
label = np.load(label_fname,allow_pickle='TRUE')
label.shape

In [ ]:
acc_list = []
std_list = []
f1_score_acc_list = []
f1_score_std_list = []
all_acc = []
all_f1_score = []
num_classes=3
folds=5
from sklearn import metrics
from tensorflow.keras.utils import to_categorical
for i in range(2):
for i in range(len(file_feat)):
    data_fname = os.path.join(mypath,file_feat[i])
    data = np.load(data_fname,allow_pickle='TRUE')
    data1=data.reshape(data.shape[0],data.shape[1]*data.shape[2])
    ##########################
    ##############################
    y_label = label
    y_label = to_categorical(y_label, num_classes)
    feat_data = data1
    print(y_label.shape)
    print(f'feat_data shape is:{feat_data.shape}')
    ############################
    kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=5)
    # create model
    for train, test in kfold.split(feat_data, y_label.argmax(1)):
        x_train = feat_data[train]
        y_train = y_label[train]
        x_in=keras.Input(shape=(x_train.shape[1],))
        x = Dense(256, activation='relu')(x_in)
        x = Dense(256, activation='relu')(x)
        x = Dense(128, activation='relu')(x)
        x = Dense(64, activation='relu')(x)
        out_layer = Dense(3, activation='softmax', name='out')(x)
        model = keras.Model(x_in, out_layer)
#         model.summary()
        # Compile model
#         loss_fn = keras.losses.SparseCategoricalCrossentropy()
#         loss_fn = keras.losses.categorical_crossentropy()
        model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=tf.keras.optimizers.Adam(),
                      metrics=['accuracy'])
        # Fit the model
        model.fit(x_train, y_train, epochs=100, batch_size=128, verbose=0)
        # evaluate the model
        x_test = feat_data[test]
        y_test = y_label[test]
        scores = model.evaluate(x_test, y_test, verbose=0)
        #############
        y_pred=model.predict(x_test)
        y_true=y_test
        y_pred=np.argmax(y_pred, axis=1)
        y_true=np.argmax(y_true, axis=1)
        # precision=metrics.precision_score(y_true, y_pred, average='macro')
        # recall=metrics.recall_score(y_true, y_pred, average='macro')
        f1_score=metrics.f1_score(y_true, y_pred, average='macro')
        all_acc.append(scores[1] * 100)
        all_f1_score.append(f1_score)
        #############
    acc_list.append(np.mean(all_acc))
    std_list.append(np.std(all_acc))
    f1_score_acc_list.append(np.mean(all_f1_score))
    f1_score_std_list.append(np.std(all_f1_score))

In [ ]:
model.summary()

In [ ]:
results=pd.DataFrame()
results['Acc_all']=acc_list
results['Std_all']=std_list
results['f1_score_acc']=f1_score_acc_list
results['f1_score_std']=f1_score_std_list
results